<a href="https://colab.research.google.com/github/anshupandey/Natural_language_Processing/blob/master/word2vec_using_gensim_%26_sentiment_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import gensim
import re

# Dataset Preparation

In [ ]:
## Downloading datasets
!wget -q https://www.dropbox.com/s/0ygoimffauvl7x5/unlabeledTrainData.tsv
!wget -q https://www.dropbox.com/s/4f1s02mh6bfjcr5/labeledTrainData.tsv

In [ ]:
df = pd.read_csv("unlabeledTrainData.tsv",delimiter="\t",quoting=3)
df.head()

,id,review
0,"""9999_0""","""Watching Time Chasers, it obvious that it was..."
1,"""45057_0""","""I saw this film about 20 years ago and rememb..."
2,"""15561_0""","""Minor Spoilers<br /><br />In New York, Joan B..."
3,"""7161_0""","""I went to see this film with a great deal of ..."
4,"""43971_0""","""Yes, I agree with everyone on this site this ..."


In [ ]:
df.shape

(50000, 2)

In [ ]:
df['review'][2]

'"Minor Spoilers<br /><br />In New York, Joan Barnard (Elvire Audrey) is informed that her husband, the archeologist Arthur Barnard (John Saxon), was mysteriously murdered in Italy while searching an Etruscan tomb. Joan decides to travel to Italy, in the company of her colleague, who offers his support. Once in Italy, she starts having visions relative to an ancient people and maggots, many maggots. After shootings and weird events, Joan realizes that her father is an international drug dealer, there are drugs hidden in the tomb and her colleague is a detective of the narcotic department. The story ends back in New York, when Joan and her colleague decide to get married with each other, in a very romantic end. Yesterday I had the displeasure of wasting my time watching this crap. The story is so absurd, mixing thriller, crime, supernatural and horror (and even a romantic end) in a non-sense way. The acting is the worst possible, highlighting the horrible performance of the beautiful El

In [ ]:
df.shape

(50000, 2)

In [ ]:
def clean_data(doc):
  doc = re.sub("<br"," ",doc)
  doc = re.sub("[^A-Za-z]"," ",doc)
  doc = " ".join([w.strip() for w in doc.strip().lower().split()])
  return doc
clean_data(df['review'][2])

'minor spoilers in new york joan barnard elvire audrey is informed that her husband the archeologist arthur barnard john saxon was mysteriously murdered in italy while searching an etruscan tomb joan decides to travel to italy in the company of her colleague who offers his support once in italy she starts having visions relative to an ancient people and maggots many maggots after shootings and weird events joan realizes that her father is an international drug dealer there are drugs hidden in the tomb and her colleague is a detective of the narcotic department the story ends back in new york when joan and her colleague decide to get married with each other in a very romantic end yesterday i had the displeasure of wasting my time watching this crap the story is so absurd mixing thriller crime supernatural and horror and even a romantic end in a non sense way the acting is the worst possible highlighting the horrible performance of the beautiful elvire audrey john saxon just gives his na

In [ ]:
df['review'] = df['review'].apply(clean_data)

#  Building the word2vec model

In [ ]:
doc_words = []
for doc in df['review']:
  doc_words.append(doc.split(' '))

In [ ]:
print(doc_words[0])

['watching', 'time', 'chasers', 'it', 'obvious', 'that', 'it', 'was', 'made', 'by', 'a', 'bunch', 'of', 'friends', 'maybe', 'they', 'were', 'sitting', 'around', 'one', 'day', 'in', 'film', 'school', 'and', 'said', 'hey', 'let', 's', 'pool', 'our', 'money', 'together', 'and', 'make', 'a', 'really', 'bad', 'movie', 'or', 'something', 'like', 'that', 'what', 'ever', 'they', 'said', 'they', 'still', 'ended', 'up', 'making', 'a', 'really', 'bad', 'movie', 'dull', 'story', 'bad', 'script', 'lame', 'acting', 'poor', 'cinematography', 'bottom', 'of', 'the', 'barrel', 'stock', 'music', 'etc', 'all', 'corners', 'were', 'cut', 'except', 'the', 'one', 'that', 'would', 'have', 'prevented', 'this', 'film', 's', 'release', 'life', 's', 'like', 'that']


In [ ]:
len(doc_words)

50000

In [ ]:
model = gensim.models.Word2Vec(doc_words,size=60,window=5,min_count=10,workers=8)

In [ ]:
model.wv['time'].shape

(60,)

In [ ]:
model.wv['anshu']

(60,)

In [ ]:
model.wv.vectors.shape

(28322, 60)

In [ ]:
model.wv.most_similar("amazing")

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('incredible', 0.8838388919830322),
 ('awesome', 0.8553487062454224),
 ('outstanding', 0.7965909838676453),
 ('exceptional', 0.7863026857376099),
 ('excellent', 0.778590977191925),
 ('fantastic', 0.7588768005371094),
 ('astounding', 0.7587637901306152),
 ('wonderful', 0.7351866364479065),
 ('brilliant', 0.7311931848526001),
 ('astonishing', 0.718397855758667)]

In [ ]:
model.wv.most_similar("yummy")

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('elke', 0.8530576229095459),
 ('sybil', 0.8473795056343079),
 ('rochelle', 0.8400182723999023),
 ('mamie', 0.8386257290840149),
 ('colleen', 0.836223840713501),
 ('jennings', 0.8339456915855408),
 ('farah', 0.8334068655967712),
 ('ditzy', 0.829382061958313),
 ('dewhurst', 0.8261468410491943),
 ('lightner', 0.8256781101226807)]

In [ ]:
model.save("imdb-vector.vec")

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:252: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


# Sentiment Analysis

In [ ]:
df = pd.read_csv("labeledTrainData.tsv",delimiter="\t",quoting=3)
df.shape

(25000, 3)

In [ ]:
df.head()

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."
3,"""3630_4""",0,"""It must be assumed that those who praised thi..."
4,"""9495_8""",1,"""Superbly trashy and wondrously unpretentious ..."


In [ ]:
df['review'] = df['review'].apply(clean_data)
df.head()

,id,sentiment,review
0,"""5814_8""",1,with all this stuff going down at the moment w...
1,"""2381_9""",1,the classic war of the worlds by timothy hines...
2,"""7759_3""",0,the film starts with a manager nicholas bell g...
3,"""3630_4""",0,it must be assumed that those who praised this...
4,"""9495_8""",1,superbly trashy and wondrously unpretentious s...


In [ ]:
df['review'][2]

## Split the dataset into train and test

In [ ]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(df['review'],df['sentiment'],test_size=0.2,random_state=5)


#### text preprocessing with tensorflow

In [ ]:
# tokenizing each review
from tensorflow.keras.preprocessing import text
#creating the tokenization object
tok = text.Tokenizer(num_words=15000)
tok.fit_on_texts(xtrain.tolist())


In [ ]:
# get the notations of tokenized words for train and test documents
xtrain = tok.texts_to_sequences(xtrain.tolist())
xtest = tok.texts_to_sequences(xtest.tolist())


In [ ]:
for doc in xtrain[:5]:
  print(len(doc))

201
90
116
418
205


In [ ]:
np.mean([len(doc) for doc in xtrain])

229.2208

In [ ]:
doc_length=300
from tensorflow.keras.preprocessing import sequence
# padding of each doc to resize all docs to same size
xtrain = sequence.pad_sequences(xtrain,maxlen=doc_length,padding='post')
xtest = sequence.pad_sequences(xtest,maxlen=doc_length,padding='post')

## Modelling part

In [ ]:
word2vec = gensim.models.Word2Vec.load("/content/imdb-vector.vec")

# embedding length for each word
vector_length=word2vec.wv.vector_size
vector_length

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:252: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


60

In [ ]:
tok.word_index

{'the': 1,
 'and': 2,
 'a': 3,
 'of': 4,
 'to': 5,
 'is': 6,
 'it': 7,
 'in': 8,
 'i': 9,
 'this': 10,
 'that': 11,
 's': 12,
 'was': 13,
 'as': 14,
 'for': 15,
 'with': 16,
 'movie': 17,
 'but': 18,
 'film': 19,
 'you': 20,
 't': 21,
 'on': 22,
 'not': 23,
 'he': 24,
 'are': 25,
 'his': 26,
 'have': 27,
 'be': 28,
 'one': 29,
 'all': 30,
 'at': 31,
 'they': 32,
 'by': 33,
 'an': 34,
 'who': 35,
 'so': 36,
 'from': 37,
 'like': 38,
 'there': 39,
 'her': 40,
 'or': 41,
 'just': 42,
 'about': 43,
 'out': 44,
 'if': 45,
 'has': 46,
 'what': 47,
 'some': 48,
 'good': 49,
 'can': 50,
 'she': 51,
 'more': 52,
 'when': 53,
 'very': 54,
 'up': 55,
 'even': 56,
 'no': 57,
 'time': 58,
 'would': 59,
 'my': 60,
 'which': 61,
 'story': 62,
 'only': 63,
 'really': 64,
 'see': 65,
 'their': 66,
 'had': 67,
 'me': 68,
 'we': 69,
 'were': 70,
 'well': 71,
 'than': 72,
 'much': 73,
 'people': 74,
 'get': 75,
 'been': 76,
 'will': 77,
 'bad': 78,
 'do': 79,
 'other': 80,
 'also': 81,
 'into': 82,
 'grea

In [ ]:
# crating a weight matrix for the words in our current local dictionary, weights are captured from
# the pretrained word2vec

weight_matrix = np.zeros((15001,60))

for word,i in sorted(tok.word_index.items(),key=lambda x:x[1]):
  print(word,i)
  if i > 15000:
    break
  if word in word2vec.wv.vocab:
    weight_matrix[i] = word2vec.wv[word]

In [ ]:
from tensorflow.keras import models,layers

In [ ]:
# modelling
model = models.Sequential()
model.add(layers.Embedding(15001,60,input_length=300,weights=[weight_matrix],
                           trainable=False))
 
model.add(layers.Flatten())
model.add(layers.Dense(500,activation='relu'))
model.add(layers.Dense(250,activation='relu'))
model.add(layers.Dense(50,activation='relu'))
model.add(layers.Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
model.fit(xtrain,ytrain,epochs=10,batch_size=100,validation_data=(xtest,ytest))

Epoch 1/10
200/200 [==============================] - 24s 120ms/step - loss: 0.5822 - accuracy: 0.7048 - val_loss: 0.4813 - val_accuracy: 0.7674
Epoch 2/10
200/200 [==============================] - 24s 120ms/step - loss: 0.3250 - accuracy: 0.8580 - val_loss: 0.5070 - val_accuracy: 0.7666
Epoch 3/10
200/200 [==============================] - 24s 119ms/step - loss: 0.1442 - accuracy: 0.9431 - val_loss: 0.7082 - val_accuracy: 0.7584
Epoch 4/10
200/200 [==============================] - 24s 120ms/step - loss: 0.0585 - accuracy: 0.9794 - val_loss: 0.9812 - val_accuracy: 0.7464
Epoch 5/10
200/200 [==============================] - 24s 120ms/step - loss: 0.0428 - accuracy: 0.9845 - val_loss: 1.1033 - val_accuracy: 0.7524
Epoch 6/10
200/200 [==============================] - 24s 120ms/step - loss: 0.0272 - accuracy: 0.9904 - val_loss: 1.1495 - val_accuracy: 0.7480
Epoch 7/10
200/200 [==============================] - 24s 120ms/step - loss: 0.0197 - accuracy: 0.9926 - val_loss: 1.2757 - val_ac

In [ ]:
model.predict(xtest[0].reshape(1,300))

array([[0.6194085]], dtype=float32)